In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
import json
from langchain.prompts import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter
import Ollama

In [ ]:
Embedding_Model = "nomic-embed-text"

In [ ]:
docLoader = WebBaseLoader("url")
document = docLoader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(document)

In [ ]:
db = FAISS.from_documents(docs, OllamaEmbeddings(model = Embedding_Model))
retriever = db.as_retriever() 

In [ ]:
def combineDocs(docs):
    context = "\n\n".join(f'Document Content : \n{doc.page_content} ]' for doc in docs)
    return context

In [ ]:
history = [' ', ' ', ' ', ' ', ' ', ' '] 
def chat(question) : 
    
    global history

    contextString = combineDocs(retriever.invoke(question))
    systemMessage = """ 
    You are a helpful assistant. You are given conversation history and a question with some context from the user. 
    Answer the question based on the information given in that context or based on the conversation history if needed."
    
    """
    message=[
                {
                    'role': 'system',
                    'content': systemMessage,
                },
                {
                    'role': 'user',
                    'content': history[-6],
                },
                {
                    'role': 'assistant',
                    'content': history[-5],
                },
                {
                    'role': 'user',
                    'content': history[-4],
                },
                {
                    'role': 'assistant',
                    'content': history[-3],
                },
                {
                    'role': 'user',
                    'content': history[-2],
                },
                {
                    'role': 'assistant',
                    'content': history[-1],
                },
                {
                    'role': 'user',
                    'content': "Here is the context : \n" + contextString,
                },{
                    'role': 'user',
                    'content': question,
                },
        ]
    
    response = ollama.chat(model = "Gemma2", messages = message)
    answer = response['message']['content']
    history.append(question)
    history.append(answer)
    
    print(answer)
    